In [33]:
import pandas as pd
import numpy as np

In [125]:
descriptors = pd.read_csv("descriptors.csv");
print('Dimensions of data frame is', descriptors.shape);

Dimensions of data frame is (1050, 247)


In [126]:
nNullVals = descriptors.isnull().sum().sum()
print('There are', nNullVals, 'null values in the dataframe')
descriptors = descriptors.dropna(axis='columns', how='all');
print('Dimensions of data frame are', descriptors.shape);

There are 31503 null values in the dataframe
Dimensions of data frame are (1050, 217)


In [127]:
pd.set_option('display.max_columns', 15)

In [152]:
descriptors= descriptors.drop(['Unnamed: 0'], axis =1)
descriptors.head()

,ActivityScore,molSIDs,XLogP,MW,LipinskiFailures,nRotB,MLogP,...,ATSc3,ATSc4,ATSc5,nHBDon,nHBAcc,bpol,apol
0,0,124897530,1.135,369.168856,0,10,2.89,...,-0.098779,0.094417,-0.105979,4,6,28.677761,57.044239
1,0,124897303,4.498,249.095378,0,2,3.11,...,-0.015299,0.003398,0.002364,0,1,15.641484,39.578516
2,0,124753561,4.554,657.230885,0,11,4.21,...,0.077083,0.204425,-0.347424,0,8,57.764452,98.933548
3,10,124753354,8.321,501.146461,1,9,2.89,...,-0.243732,0.222481,-0.064752,2,4,40.743589,70.138411
4,0,121286038,4.515,637.224657,0,10,4.32,...,0.081751,0.016048,0.040308,2,6,53.306245,97.709755


In [64]:
scoreData = pd.read_csv("data.csv");
sid_activityScore = scoreData.loc[5:,['PUBCHEM_SID','PUBCHEM_ACTIVITY_SCORE']];

/Users/daraakdag/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,8,9,10,11,13,14,15,16,17,18,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [66]:
sid_activityScore.head()

,PUBCHEM_SID,PUBCHEM_ACTIVITY_SCORE
5,842121.0,0.0
6,842122.0,0.0
7,842123.0,0.0
8,842124.0,0.0
9,842125.0,0.0


In [67]:
# Convert floating point values to integers. 
sid_activityScore['PUBCHEM_SID'] = sid_activityScore['PUBCHEM_SID'].astype(np.int64)
sid_activityScore['PUBCHEM_ACTIVITY_SCORE'] = sid_activityScore['PUBCHEM_ACTIVITY_SCORE'].astype(np.int64)

In [68]:
sid_activityScore.head()

,PUBCHEM_SID,PUBCHEM_ACTIVITY_SCORE
5,842121,0
6,842122,0
7,842123,0
8,842124,0
9,842125,0


In [ ]:
# Insert activity score column with values 0
descriptors.insert(0,'ActivityScore',int(0))

In [153]:
descriptors.head()

,ActivityScore,molSIDs,XLogP,MW,LipinskiFailures,nRotB,MLogP,...,ATSc3,ATSc4,ATSc5,nHBDon,nHBAcc,bpol,apol
0,0,124897530,1.135,369.168856,0,10,2.89,...,-0.098779,0.094417,-0.105979,4,6,28.677761,57.044239
1,0,124897303,4.498,249.095378,0,2,3.11,...,-0.015299,0.003398,0.002364,0,1,15.641484,39.578516
2,0,124753561,4.554,657.230885,0,11,4.21,...,0.077083,0.204425,-0.347424,0,8,57.764452,98.933548
3,10,124753354,8.321,501.146461,1,9,2.89,...,-0.243732,0.222481,-0.064752,2,4,40.743589,70.138411
4,0,121286038,4.515,637.224657,0,10,4.32,...,0.081751,0.016048,0.040308,2,6,53.306245,97.709755


In [144]:
# Lookup activity scores for each molSID of descriptors matrix in the sid_activityScore dataframe 
# and insert activity score into descriptor matrix. 
for idx,val in enumerate(descriptors['molSIDs']):
    temp = sid_activityScore.loc[sid_activityScore['PUBCHEM_SID']==val];
    score = temp.iloc[0][1]
    descriptors.at[idx,'ActivityScore'] = score;


In [146]:
descriptors.tail()

,Unnamed: 0,ActivityScore,molSIDs,XLogP,MW,LipinskiFailures,nRotB,...,ATSc3,ATSc4,ATSc5,nHBDon,nHBAcc,bpol,apol
1045,1046,42,843609,3.454,321.114713,0,8,...,0.116596,0.019901,-0.292837,0,4,31.140933,47.675067
1046,1047,0,843396,2.894,334.121195,0,3,...,0.466517,-0.486335,0.420218,0,6,34.433726,47.746274
1047,1048,0,843202,1.316,423.179421,0,6,...,-0.151360,0.434316,-0.400282,1,7,38.656175,64.459825
1048,1049,10,842577,2.752,271.024932,0,4,...,-0.030790,0.078024,-0.033099,1,3,19.271070,34.726930
1049,1050,10,842218,3.820,368.140389,0,4,...,-0.115749,0.072170,0.032923,0,4,28.295347,56.584653
